In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'

In [ ]:
def process_frame(current_frame: np.ndarray, lower_range:np.ndarray = None, upper_range:np.ndarray = None) -> np.ndarray:
    """Process single frame and return new frame with detected objects."""

    height, width, _ = current_frame.shape
    original_image: np.ndarray = current_frame.copy()
    
    hsv_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)
    
    if lower_range is None:
        lower_range = np.array([0, 90, 160])
        
    if upper_range is None:
        upper_range = np.array([30, 255, 255])

    binary_image = cv2.inRange(hsv_image, lower_range, upper_range)
    median = cv2.medianBlur(binary_image, 5)
    eroded = cv2.erode(median, np.ones((3, 3)))
    dilated = cv2.dilate(eroded, np.ones((5, 5)))
    dilated = cv2.erode(dilated, np.ones((5, 5)))
    # blurred = cv2.GaussianBlur(median, (3, 3), 0)
    # opened_image = cv2.morphologyEx(blurred, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
    
    processed = dilated

    # Add a black rectangle to the upper right quarter of the image opened_image
    # opened_image[0: height // 2,
    #              width // 2: width] = 0

    # Find contours and add rectangles to the original image

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(processed)
    
    labeled_image = original_image.copy()
    for i in range(1, num_labels):
        areas = stats[:, -1]  # Get the areas of all labels
        top_labels = np.argsort(areas)[::-1][:6]  # Get the indices of the top 5 labels with the largest areas

        for i in top_labels:
            x, y, w, h, area = stats[i]
            if 8 < area < 150:
                cv2.rectangle(labeled_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(labeled_image, f'{area}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return labeled_image, original_image, binary_image, processed


def process_video(file: str, lower = None, upper = None) -> None:
    """Process video file and display it with detected objects."""
    capture = cv2.VideoCapture(file)

    while capture.isOpened():
        has_next_frame, current_frame = capture.read()
        if not has_next_frame:
            break

        labeled, original, binary, processed = process_frame(current_frame, lower, upper)
        cv2.imshow('Labels', labeled)
        cv2.imshow('Original', original)
        cv2.imshow('Processed', processed)
        cv2.imshow('Binary', binary)
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()

In [ ]:
lower_range = np.array([0, 90, 160])
upper_range = np.array([30, 255, 255])

yellow_lower = np.array([0, 0, 230])
yellow_upper = np.array([35, 255, 255])

red_lower = np.array([175, 86, 0])
red_upper = np.array([179, 255, 255])

In [ ]:
process_video('mats/video_2.mp4', yellow_lower, yellow_upper)